In [ ]:
!pip install folium requests

In [ ]:
import requests

def get_coordinates(city):
    url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    response = requests.get(url).json()
    if "results" in response and len(response["results"]) > 0:
        result = response["results"][0]
        lat, lon = result["latitude"], result["longitude"]
        print(f"{city}: ({lat}, {lon})")
        return lat, lon
    else:
        print("City not found.")
        return None, None

city = 'Helsinki'
lat, lon = get_coordinates(city)


Helsinki: (60.16952, 24.93545)


In [ ]:
def get_weather(lat, lon):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    data = requests.get(url).json()
    current = data["current_weather"]
    temp = current["temperature"]
    wind = current["windspeed"]
    weather_code = current["weathercode"]
    print(f"Temp: {temp}°C | Wind: {wind} km/h")
    return temp, wind, weather_code

weather = get_weather(lat, lon)


Temp: 7.9°C | Wind: 18.4 km/h


In [ ]:
def get_air_quality(lat, lon):
    url = (
        "https://air-quality-api.open-meteo.com/v1/air-quality"
        f"?latitude={lat}&longitude={lon}"
        "&hourly=pm10,pm2_5,carbon_monoxide,ozone,nitrogen_dioxide,sulphur_dioxide,european_aqi"
    )
    data = requests.get(url).json()
    hourly = data["hourly"]
    aqi = hourly["european_aqi"][-1]
    pm25 = hourly["pm2_5"][-1]
    pm10 = hourly["pm10"][-1]
    no2 = hourly["nitrogen_dioxide"][-1]
    print(f"European AQI: {aqi} | PM2.5: {pm25} µg/m³ | PM10: {pm10} µg/m³ | NO₂: {no2} µg/m³")
    return aqi, pm25, pm10, no2

aqi, pm25, pm10, no2 = get_air_quality(lat, lon)


European AQI: 32 | PM2.5: 2.1 µg/m³ | PM10: 3.1 µg/m³ | NO₂: 2.2 µg/m³


In [ ]:
import folium

def get_aqi_color(aqi):
    if aqi <= 1:
        return "green"
    elif aqi == 2:
        return "yellow"
    elif aqi == 3:
        return "orange"
    elif aqi == 4:
        return "red"
    else:
        return "purple"

m = folium.Map(location=[lat, lon], zoom_start=10)

popup_html = f"""
<b>{city}</b><br>
Temperature: {weather[0]}°C<br>
Wind: {weather[1]} km/h<br>
European AQI: {aqi}<br>
PM2.5: {pm25} µg/m³<br>
PM10: {pm10} µg/m³<br>
NO₂: {no2} µg/m³
"""

folium.Marker(
    location=[lat, lon],
    popup=folium.Popup(popup_html, max_width=250),
    icon=folium.Icon(color=get_aqi_color(aqi))
).add_to(m)

m
